In [1]:
import praw
import prawcore
import json
from datetime import datetime, timezone
from dotenv import load_dotenv, dotenv_values 

import requests

from tqdm import tqdm
import time

import pandas as pd
import os
from reddit_scraper import RedditScraper
load_dotenv()

True

In [2]:
client_id = os.getenv("CLIENT_ID")
client_secret=os.getenv("CLIENT_SECRET")
user_agent=os.getenv("USER_AGENT")

In [3]:
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent,
)

In [4]:
output = reddit.multireddit(redditor="samuraisam", name="programming").new()

In [ ]:
for outp in output:
    

In [4]:
def date_to_epoch(date):
    return int(datetime.strptime(date, "%Y-%m-%d").timestamp())

In [ ]:
def scrape_reddit_user_comments(username, subreddit, start_date, end_date):
    start_epoch = date_to_epoch(start_date)
    end_epoch = date_to_epoch(end_date)
    url = f"https://api.pushshift.io/reddit/comment/search"
    params = {
        "author": username,
        "subreddit": subreddit,
        "after": start_epoch,
        "before": end_epoch,
        "size": 100,  # Fetch 100 comments at a time (max limit)
    }
    comments = []
    while True:
        try:
            response = requests.get(url, params=params)
            if response.status_code != 200:
                print(f"Error: {response.status_code}")
                break
            data = response.json().get('data', [])
            if not data:
                break
            for comment in data:
                comments.append({
                    "id": comment["id"],
                    "subreddit": comment["subreddit"],
                    "body": comment["body"],
                    "created_utc": datetime.utcfromtimestamp(comment["created_utc"]).strftime('%Y-%m-%d %H:%M:%S')
                })
            # Use the last comment's created_utc as the next starting point
            params["after"] = data[-1]["created_utc"]
        except prawcore.exceptions.TooManyRequests as e:
            print(f"Rate limit hit for comments of post {username}. Sleeping for {e.retry_after} seconds.")
            time.sleep(61.0)  # Sleep for the time suggested by the error
            continue  # Retry the request after waiting 
    return comments

In [15]:
# Example usage
username = "1barefootmaniac"
subreddit = "books"
start_date = "2024-01-01"
end_date = "2024-12-01"
comments = scrape_reddit_user_comments(username, subreddit, start_date, end_date)


Error: 403


In [ ]:
for comment in comments:
    print(f"{comment['created_utc']} | {comment['subreddit']} | {comment['body']}\n")